# Convert Spectra fits file to Pandas

- We use the quality cut produced in **SelectGoodObservationfrmQualityFlag.ipynb**


- Purpose is to write good observations in the output excel file

- author : Sylvie Dagoret-Campagne
- creation date : September 30th 2020


In [14]:
#########################################################################################################
# Study grey attenuation
#########################################################################################################

import os
from os import listdir
from os.path import isfile, join
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import re

import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

import numpy as np

from astropy.time import Time

if not 'workbookDir' in globals():
    workbookDir = os.getcwd()
print('workbookDir: ' + workbookDir)

spectractordir=workbookDir+"/../../Spectractor"
print('spectractordir: ' + spectractordir)
toolsdir=workbookDir+"/../common_tools"
print("toolsdir:",toolsdir)



import sys
sys.path.append(workbookDir)
sys.path.append(spectractordir)
sys.path.append(os.path.dirname(workbookDir))
sys.path.append(toolsdir)

from astropy.table import Table, Row


from spectractor import parameters
from spectractor.extractor.extractor import Spectractor
from spectractor.logbook import LogBook
from spectractor.extractor.dispersers import *
from spectractor.extractor.spectrum import *
from spectractor.tools import ensure_dir

from libatmscattering import *


plt.rcParams["axes.labelsize"]="large"
plt.rcParams["axes.linewidth"]=2.0
plt.rcParams["xtick.major.size"]=8
plt.rcParams["ytick.major.size"]=8
plt.rcParams["ytick.minor.size"]=5
plt.rcParams["xtick.labelsize"]="large"
plt.rcParams["ytick.labelsize"]="large"

plt.rcParams["figure.figsize"]=(20,20)
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.titleweight'] = 'bold'
#plt.rcParams['axes.facecolor'] = 'blue'
plt.rcParams['xtick.direction'] = 'out'
plt.rcParams['ytick.direction'] = 'out'
plt.rcParams['lines.markeredgewidth'] = 0.3 # the line width around the marker symbol
plt.rcParams['lines.markersize'] = 5  # markersize, in points
plt.rcParams['grid.alpha'] = 0.75 # transparency, between 0.0 and 1.0
plt.rcParams['grid.linestyle'] = '-' # simple line
plt.rcParams['grid.linewidth'] = 0.4 # in points

WLMIN = 380.0
WLMAX = 1000.0
NBWLBIN = 62
WLBINWIDTH = (WLMAX - WLMIN) / float(NBWLBIN)

WLMINBIN = np.arange(WLMIN, WLMAX, WLBINWIDTH)
WLMAXBIN = np.arange(WLMIN + WLBINWIDTH, WLMAX + WLBINWIDTH, WLBINWIDTH)
WLMEANBIN=(WLMINBIN + WLMAXBIN)/2.



print('WLMINBIN..................................=', WLMINBIN.shape, WLMINBIN)
print('WLMAXBIN..................................=', WLMAXBIN.shape, WLMAXBIN)
print('NBWLBIN...................................=', NBWLBIN)
print('WLBINWIDTH................................=', WLBINWIDTH)

#---------------------------------------------------------------------
def GetWLBin(wl):
    """

    :param wl: wavelength scalar
    :return: index
    """

    set_ibin = np.where(np.logical_and(WLMINBIN <= wl, WLMAXBIN > wl))[0]

    if len(set_ibin)==1:
        return set_ibin[0]
    else:
        return -1
#---------------------------------------------------------------------
def GETWLLabels():

    all_labels=[]
    for idx in np.arange(NBWLBIN):
        label="$\lambda$ : {:3.0f}-{:3.0f} nm".format(WLMINBIN[idx], WLMAXBIN[idx])
        all_labels.append(label)
    all_labels=np.array(all_labels)
    return all_labels
#------------------------------------------------------------------------

WLLABELS=GETWLLabels()


tablesdir="tables_clean"


workbookDir: /Users/dagoret/MacOSX/GitHub/LSST/SpectractorAnaAtm19/ana_20190215_HD116405_Filtre_None
spectractordir: /Users/dagoret/MacOSX/GitHub/LSST/SpectractorAnaAtm19/ana_20190215_HD116405_Filtre_None/../../Spectractor
toolsdir: /Users/dagoret/MacOSX/GitHub/LSST/SpectractorAnaAtm19/ana_20190215_HD116405_Filtre_None/../common_tools
WLMINBIN..................................= (62,) [380. 390. 400. 410. 420. 430. 440. 450. 460. 470. 480. 490. 500. 510.
 520. 530. 540. 550. 560. 570. 580. 590. 600. 610. 620. 630. 640. 650.
 660. 670. 680. 690. 700. 710. 720. 730. 740. 750. 760. 770. 780. 790.
 800. 810. 820. 830. 840. 850. 860. 870. 880. 890. 900. 910. 920. 930.
 940. 950. 960. 970. 980. 990.]
WLMAXBIN..................................= (62,) [ 390.  400.  410.  420.  430.  440.  450.  460.  470.  480.  490.  500.
  510.  520.  530.  540.  550.  560.  570.  580.  590.  600.  610.  620.
  630.  640.  650.  660.  670.  680.  690.  700.  710.  720.  730.  740.
  750.  760.  770.  780.  79

# Read the selection of files

In [15]:
#quality_flag_file="images-pic-15fev19-cut.xlsx"
input_selected_files = "selected_images-pic-15fev19-cut.xlsx" 

In [16]:
dfgood = pd.read_excel(input_selected_files)

In [17]:
dfgood

,Unnamed: 0,date,file,remarques,FWHM (pix),Xstar1,Ystar1,Xstar2,Ystar2,X1,Y1
0,215,Début Bouguer,T1M_20190215_225550_730_HD116405_Filtre_None_b...,OK,NaN,132.823511,219.841672,416.823511,89.841672,133.208557,220.877829
1,216,Avec masque,T1M_20190215_225626_007_HD116405_Filtre_None_b...,OK,NaN,133.142897,216.101870,417.142897,86.101870,133.400674,217.614830
2,217,Avec tilt 1°,T1M_20190215_225701_284_HD116405_Filtre_None_b...,OK,NaN,132.873758,216.523630,416.873758,86.523630,133.268162,217.958401
3,218,NaN,T1M_20190215_225736_562_HD116405_Filtre_None_b...,OK,NaN,183.426957,172.901954,467.426957,42.901954,183.596980,174.986783
4,219,NaN,T1M_20190215_225811_838_HD116405_Filtre_None_b...,OK,NaN,179.930506,173.063874,463.930506,43.063874,180.257505,174.551341
5,220,NaN,T1M_20190215_225847_115_HD116405_Filtre_None_b...,OK,NaN,178.533028,171.781995,462.533028,41.781995,178.797906,173.034297
6,221,NaN,T1M_20190215_225922_399_HD116405_Filtre_None_b...,OK,NaN,178.265474,171.770764,462.265474,41.770764,178.520856,173.230098
7,222,NaN,T1M_20190215_225957_677_HD116405_Filtre_None_b...,OK,NaN,176.188132,168.321202,460.188132,38.321202,176.219027,169.545033
8,223,NaN,T1M_20190215_230032_956_HD116405_Filtre_None_b...,OK,NaN,172.667399,166.385768,456.667399,36.385768,173.046606,167.726341
9,224,NaN,T1M_20190215_230108_233_HD116405_Filtre_None_b...,OK,NaN,171.714977,164.965966,455.714977,34.965966,171.784770,166.292867


In [18]:
dfgood["file"][0]

'T1M_20190215_225550_730_HD116405_Filtre_None_bin1x1.1.fit'

In [19]:
list_of_goodfiles=dfgood["file"]

In [20]:
list_of_rootfilename = [ file.split("_bin")[0] for file in list_of_goodfiles]

In [21]:
list_of_rootfilename

['T1M_20190215_225550_730_HD116405_Filtre_None',
 'T1M_20190215_225626_007_HD116405_Filtre_None',
 'T1M_20190215_225701_284_HD116405_Filtre_None',
 'T1M_20190215_225736_562_HD116405_Filtre_None',
 'T1M_20190215_225811_838_HD116405_Filtre_None',
 'T1M_20190215_225847_115_HD116405_Filtre_None',
 'T1M_20190215_225922_399_HD116405_Filtre_None',
 'T1M_20190215_225957_677_HD116405_Filtre_None',
 'T1M_20190215_230032_956_HD116405_Filtre_None',
 'T1M_20190215_230108_233_HD116405_Filtre_None',
 'T1M_20190215_230143_511_HD116405_Filtre_None',
 'T1M_20190215_230218_787_HD116405_Filtre_None',
 'T1M_20190215_230254_099_HD116405_Filtre_None',
 'T1M_20190215_230329_374_HD116405_Filtre_None',
 'T1M_20190215_230404_652_HD116405_Filtre_None',
 'T1M_20190215_230439_930_HD116405_Filtre_None',
 'T1M_20190215_230515_206_HD116405_Filtre_None',
 'T1M_20190215_230550_481_HD116405_Filtre_None',
 'T1M_20190215_230625_768_HD116405_Filtre_None',
 'T1M_20190215_230701_044_HD116405_Filtre_None',
 'T1M_20190215_23073

In [22]:
def test_if_good_file(filename,listrootfilename):
    """
    """
    
    flag_good=False
    
    for goodfile in listrootfilename:
        stringsearch=f"^{goodfile}.*"
        if re.search(stringsearch,filename):
            flag_good=True
            break
            
    return flag_good

# Loop on spectra

In [23]:
ensure_dir(tablesdir)

In [24]:
#-------------------------------------------------------------------------
#
# MAIN()
#
#---------------------------------------------------

if __name__ == "__main__":

    #####################
    # 1) Configuration
    ######################

    parameters.VERBOSE = True
    parameters.DEBUG = True

    #thedate="20190214"
    thedate = "20190215"

    #output_directory = "output/" + thedate
    #output_directory = "/Users/dagoret/DATA/PicDuMidiFev2019/spectractor_spectra/" + thedate
    #output_directory = "/Users/dagoret/DATA/PicDuMidiFev2019/spectractor_output_deco/"+ thedate
    #output_directory = "/Users/dagoret/DATA/PicDuMidiFev2019/spectractor_output_deco2/" + thedate
    # dernière production sur deco
    output_directory = "/Users/dagoret/DATA/PicDuMidiFev2019/spectractor_output_prod4/" + thedate

    parameters.VERBOSE = True
    parameters.DISPLAY = True




    ############################
    # 2) Get the config
    #########################

    config = spectractordir+"/config/picdumidi.ini"

    my_logger = set_logger(__name__)
    my_logger.info('\n\tconvertSpectraToPandas')
    # Load config file
    load_config(config)



    ############################
    # 3) Get Spectra filelist
    #########################


    # get all files
    onlyfiles = [f for f in listdir(output_directory) if isfile(join(output_directory, f))]
    onlyfiles = np.array(onlyfiles)

    # sort files
    sortedindexes = np.argsort(onlyfiles)
    onlyfiles = onlyfiles[sortedindexes]

    # get only _spectrum.fits file
    onlyfilesspectrum = []
    onlyfilesspectrum_qualityflag = []
    
    idx=0
    
    for file_name in onlyfiles:
        if re.search("^T.*_spectrum.fits$", file_name):
            # check if other files exits
            
            quality_flag=False
            
            if test_if_good_file(file_name,list_of_rootfilename):
                print("*************************************************************************************")
                print("*                ************  GOOD FILE = ",file_name)
                print("*************************************************************************************")
                quality_flag=True
            else:
                print("*************************************************************************************")
                print("*                >>>>>>>>>>>>  BAD FILE = ",file_name)
                print("*************************************************************************************")
            
            

            filetype = file_name.split('.')[-1]

            output_filename = os.path.basename(file_name)
            output_filename = os.path.join(output_directory, output_filename)
            output_filename_spectrogram = output_filename.replace('spectrum', 'spectrogram')
            output_filename_psf = output_filename.replace('spectrum.fits', 'table.csv')



            # go to next simulation if output files already exists
            if os.path.isfile(output_filename) and os.path.isfile(output_filename_spectrogram) and os.path.isfile(
                        output_filename_psf):
                filesize = os.stat(output_filename).st_size
                print(">>>>> output filename : {} already exists with size {} ".format(output_filename,filesize))

                filesize = os.stat(output_filename_spectrogram).st_size
                print(">>>>> output filename : {} already exists with size {} ".format(output_filename_spectrogram,filesize))

                filesize = os.stat(output_filename_psf).st_size
                print(">>>>> output filename : {} already exists with size {} ".format(output_filename_psf,filesize))

                onlyfilesspectrum.append(re.findall("(^T.*_spectrum.fits$)", file_name)[0])

                onlyfilesspectrum_qualityflag.append(quality_flag)


    # sort again all the files
    onlyfilesspectrum = np.array(onlyfilesspectrum)
    onlyfilesspectrum_qualityflag = np.array(onlyfilesspectrum_qualityflag)
    
    sortedindexes = np.argsort(onlyfilesspectrum)
    onlyfilesspectrum = onlyfilesspectrum[sortedindexes]                        # spectra
    onlyfilesspectrum_qualityflag=onlyfilesspectrum_qualityflag[sortedindexes]  # flag

    #get basename of files for later use (to check if _table.csv and _spectrogram.fits exists
    onlyfilesbasename=[]
    for f in onlyfilesspectrum:
        onlyfilesbasename.append(re.findall("(^T.*)_spectrum.fits$",f)[0])


    basenamecut=[]
    for f in onlyfilesspectrum:
        basenamecut.append(f.split("_HD")[0])


    #############################################
    # 3) Read fits file
    ##########################################

    # parameters
    NBSPEC = len(sortedindexes)

    print('NBSPEC....................................= ', NBSPEC)


    #assert False


    all_indexes=[]
    all_airmass=[]
    all_flag=[]
    all_flux=[]
    all_flux_err=[]
    all_lambdas=[]
    all_mag=[]
    all_errmag=[]
    all_abs=[]
    all_errabs=[]
    all_dt=[]
    all_badidx=[]
    all_badfn=[]
    all_filename=[]

    #----------------------------------
    # Extract spectra information from files
    # compute magnitudes
    #-----------------------------
    count=-1
    # loop on spectra
    for idx in np.arange(0, NBSPEC):
        if idx==322:
            print("SKIP bad file",onlyfilesspectrum[idx] )
            continue

        count += 1


        strnum = f'{count:03}'
        output_pdfile = "attenuationdata_clean{}.csv".format(strnum)





        print(" read file {}) : {}".format(idx,onlyfilesspectrum[idx]))

        fullfilename = os.path.join(output_directory, onlyfilesspectrum[idx])
        #try:
        if 1:
            hdu = fits.open(fullfilename)
            header=hdu[0].header

            am=header["AIRMASS"]
            date=header["DATE-OBS"]
            if idx==0:
                T0=t = Time(date, format='isot', scale='utc')
            T=Time(date, format='isot', scale='utc')
            DT=(T-T0).sec/(3600.0)  # time in hours

            data=hdu[0].data

            # extract wavelength, spectrum and errors
            wavelength=data[0,:]
            spec = data[1,:]
            err=data[2,: ]

            if(len(wavelength)==0 or len(spec)==0 or len(err)==0):
                print(">>>>>>>>>>>>>>  Empty file ",idx,")::",onlyfilesspectrum[idx] )
                print(">>>>>>>>>>>>>>>>>>>>>>>>>>>",len(wavelength)," , ",len(spec0), " , ",len(err))



            # sort the wavelengths
            wl_sorted_idx=np.argsort(wavelength)

            wl=wavelength[wl_sorted_idx]
            fl=spec[wl_sorted_idx]
            errfl=err[wl_sorted_idx]
            wlbins = [GetWLBin(w) for w in wl]
            wlbins = np.array(wlbins)


            # defines good measurmements as flux >0 and wavelength in selected bons
            goodpoints=np.where(np.logical_and(fl != 0, wlbins != -1))

            if(len(goodpoints)==0):
                print(">>>>>>>>>>>>>>  No Good points  ", idx, ")::", onlyfilesspectrum[idx])
                print(">>>>>>>>>>>>>>  No Good points  ", "wl = ",wl)
                print(">>>>>>>>>>>>>>  No Good points  ", "fl = ", fl)
                print(">>>>>>>>>>>>>>  No Good points  ", "errfl = ", errfl)



            # keep good points (wl,flux)
            wl=wl[goodpoints]
            fl=fl[goodpoints]
            errfl=errfl[goodpoints]


            # convert flux into magnitudes for each wavelength
            mag = -2.5 * np.log10(fl)
            errmag = errfl/fl

            #compute effective slant Rayleigh optical depth (not the vertical optical depth)
            od_adiab = RayOptDepth_adiabatic(wl,altitude=2890.5, costh=1./am)  # Rayleigh optical depth

            # absorption magnitude corrected from Rayleigh attenuation
            abs=mag-2.5/np.log(10.)*od_adiab
            errabs=errmag

            absmin = abs.min()
            absmax = abs.max()


            df=pd.DataFrame()

            nwl=len(wl)
            df["count"]=np.full(nwl,count)
            df["airmass"] = np.full(nwl,am)
            df["time"] = np.full(nwl,DT)
            df["wavelength"] = wl
            df["flux"] = fl
            df["errflux"] = errfl
            df["mag"] = mag
            df["errmag"] = errmag
            df["abs"] = abs
            df["errabs"] = errabs

            if onlyfilesspectrum_qualityflag[idx]:
                df.to_csv(os.path.join(tablesdir,output_pdfile))
            else:
                print("*************************************************************************************")
                print("*    >>>> Reject bad quality file {}) : {}".format(idx,onlyfilesspectrum[idx]))
                print("*************************************************************************************")
                


2020-09-30 18:10:36  __main__   <module>             INFO   
	convertSpectraToPandas


Section: paths
x holo_dir: extractor/dispersers/	 => parameters.HOLO_DIR: /Users/dagoret/MacOSX/GitHub/LSST/SpectractorAnaAtm19/ana_20190215_HD116405_Filtre_None/../../Spectractor/spectractor/extractor/dispersers/	 <class 'str'>
x throughput_dir: simulation/CTIOThroughput/	 => parameters.THROUGHPUT_DIR: /Users/dagoret/MacOSX/GitHub/LSST/SpectractorAnaAtm19/ana_20190215_HD116405_Filtre_None/../../Spectractor/spectractor/simulation/CTIOThroughput/	 <class 'str'>
Section: instrument
x obs_name: PICDUMIDI	 => parameters.OBS_NAME: PICDUMIDI	 <class 'str'>
x obs_altitude: 2.8905	 => parameters.OBS_ALTITUDE: 2.8905	 <class 'float'>
x obs_longitude: 0 08 34.00	 => parameters.OBS_LONGITUDE: 0 08 34.00	 <class 'str'>
x obs_latitude: 42 56 11.00	 => parameters.OBS_LATITUDE: 42 56 11.00	 <class 'str'>
x obs_diameter: 1.0	 => parameters.OBS_DIAMETER: 1.0 m	 <class 'astropy.units.quantity.Quantity'>
x obs_epoch: J2000.0	 => parameters.OBS_EPOCH: J2000.0	 <class 'str'>
x obs_transmission_systematics:

>>>>> output filename : /Users/dagoret/DATA/PicDuMidiFev2019/spectractor_output_prod4/20190215/T1M_20190216_003538_871_HD116405_Filtre_None_bin1x1.36_red_spectrogram.fits already exists with size 36501120 
>>>>> output filename : /Users/dagoret/DATA/PicDuMidiFev2019/spectractor_output_prod4/20190215/T1M_20190216_003538_871_HD116405_Filtre_None_bin1x1.36_red_table.csv already exists with size 609300 
*************************************************************************************
*                ************  GOOD FILE =  T1M_20190216_003614_149_HD116405_Filtre_None_bin1x1.37_red_spectrum.fits
*************************************************************************************
>>>>> output filename : /Users/dagoret/DATA/PicDuMidiFev2019/spectractor_output_prod4/20190215/T1M_20190216_003614_149_HD116405_Filtre_None_bin1x1.37_red_spectrum.fits already exists with size 51840 
>>>>> output filename : /Users/dagoret/DATA/PicDuMidiFev2019/spectractor_output_prod4/20190215/T1M_20190216_

>>>>> output filename : /Users/dagoret/DATA/PicDuMidiFev2019/spectractor_output_prod4/20190215/T1M_20190216_024925_536_HD116405_Filtre_None_bin1x1.230_red_spectrum.fits already exists with size 48960 
>>>>> output filename : /Users/dagoret/DATA/PicDuMidiFev2019/spectractor_output_prod4/20190215/T1M_20190216_024925_536_HD116405_Filtre_None_bin1x1.230_red_spectrogram.fits already exists with size 33261120 
>>>>> output filename : /Users/dagoret/DATA/PicDuMidiFev2019/spectractor_output_prod4/20190215/T1M_20190216_024925_536_HD116405_Filtre_None_bin1x1.230_red_table.csv already exists with size 583021 
*************************************************************************************
*                ************  GOOD FILE =  T1M_20190216_025000_829_HD116405_Filtre_None_bin1x1.231_red_spectrum.fits
*************************************************************************************
>>>>> output filename : /Users/dagoret/DATA/PicDuMidiFev2019/spectractor_output_prod4/20190215/T1M_20190

*************************************************************************************
*                ************  GOOD FILE =  T1M_20190216_045349_846_HD116405_Filtre_None_bin1x1.424_red_spectrum.fits
*************************************************************************************
>>>>> output filename : /Users/dagoret/DATA/PicDuMidiFev2019/spectractor_output_prod4/20190215/T1M_20190216_045349_846_HD116405_Filtre_None_bin1x1.424_red_spectrum.fits already exists with size 51840 
>>>>> output filename : /Users/dagoret/DATA/PicDuMidiFev2019/spectractor_output_prod4/20190215/T1M_20190216_045349_846_HD116405_Filtre_None_bin1x1.424_red_spectrogram.fits already exists with size 38548800 
>>>>> output filename : /Users/dagoret/DATA/PicDuMidiFev2019/spectractor_output_prod4/20190215/T1M_20190216_045349_846_HD116405_Filtre_None_bin1x1.424_red_table.csv already exists with size 623655 
*************************************************************************************
*                *

...

 read file 151) : T1M_20190216_004242_211_HD116405_Filtre_None_bin1x1.48_red_spectrum.fits
 read file 152) : T1M_20190216_004317_489_HD116405_Filtre_None_bin1x1.49_red_spectrum.fits
 read file 153) : T1M_20190216_004352_766_HD116405_Filtre_None_bin1x1.50_red_spectrum.fits
 read file 154) : T1M_20190216_004428_043_HD116405_Filtre_None_bin1x1.51_red_spectrum.fits
 read file 155) : T1M_20190216_004503_321_HD116405_Filtre_None_bin1x1.52_red_spectrum.fits
 read file 156) : T1M_20190216_004538_605_HD116405_Filtre_None_bin1x1.53_red_spectrum.fits
 read file 157) : T1M_20190216_004613_884_HD116405_Filtre_None_bin1x1.54_red_spectrum.fits
 read file 158) : T1M_20190216_004649_162_HD116405_Filtre_None_bin1x1.55_red_spectrum.fits
 read file 159) : T1M_20190216_004724_440_HD116405_Filtre_None_bin1x1.56_red_spectrum.fits
 read file 160) : T1M_20190216_004759_718_HD116405_Filtre_None_bin1x1.57_red_spectrum.fits
 read file 161) : T1M_20190216_004834_996_HD116405_Filtre_None_bin1x1.58_red_spectrum.fits

 read file 219) : T1M_20190216_014255_317_HD116405_Filtre_None_bin1x1.130_red_spectrum.fits
 read file 220) : T1M_20190216_014330_617_HD116405_Filtre_None_bin1x1.131_red_spectrum.fits
 read file 221) : T1M_20190216_014405_917_HD116405_Filtre_None_bin1x1.132_red_spectrum.fits
 read file 222) : T1M_20190216_014441_242_HD116405_Filtre_None_bin1x1.133_red_spectrum.fits
*************************************************************************************
*    >>>> Reject bad quality file 222) : T1M_20190216_014441_242_HD116405_Filtre_None_bin1x1.133_red_spectrum.fits
*************************************************************************************
 read file 223) : T1M_20190216_014516_541_HD116405_Filtre_None_bin1x1.134_red_spectrum.fits
*************************************************************************************
*    >>>> Reject bad quality file 223) : T1M_20190216_014516_541_HD116405_Filtre_None_bin1x1.134_red_spectrum.fits
****************************************************

 read file 259) : T1M_20190216_031442_971_HD116405_Filtre_None_bin1x1.273_red_spectrum.fits
 read file 260) : T1M_20190216_031518_263_HD116405_Filtre_None_bin1x1.274_red_spectrum.fits
 read file 261) : T1M_20190216_031553_551_HD116405_Filtre_None_bin1x1.275_red_spectrum.fits
 read file 262) : T1M_20190216_032830_153_HD116405_Filtre_None_bin1x1.292_red_spectrum.fits
*************************************************************************************
*    >>>> Reject bad quality file 262) : T1M_20190216_032830_153_HD116405_Filtre_None_bin1x1.292_red_spectrum.fits
*************************************************************************************
 read file 263) : T1M_20190216_033126_585_HD116405_Filtre_None_bin1x1.297_red_spectrum.fits
*************************************************************************************
*    >>>> Reject bad quality file 263) : T1M_20190216_033126_585_HD116405_Filtre_None_bin1x1.297_red_spectrum.fits
****************************************************

 read file 292) : T1M_20190216_040644_146_HD116405_Filtre_None_bin1x1.357_red_spectrum.fits
*************************************************************************************
*    >>>> Reject bad quality file 292) : T1M_20190216_040644_146_HD116405_Filtre_None_bin1x1.357_red_spectrum.fits
*************************************************************************************
 read file 293) : T1M_20190216_040719_441_HD116405_Filtre_None_bin1x1.358_red_spectrum.fits
*************************************************************************************
*    >>>> Reject bad quality file 293) : T1M_20190216_040719_441_HD116405_Filtre_None_bin1x1.358_red_spectrum.fits
*************************************************************************************
 read file 294) : T1M_20190216_041051_213_HD116405_Filtre_None_bin1x1.364_red_spectrum.fits
*************************************************************************************
*    >>>> Reject bad quality file 294) : T1M_20190216_041051_213

 read file 340) : T1M_20190216_050128_547_HD116405_Filtre_None_bin1x1.437_red_spectrum.fits
 read file 341) : T1M_20190216_050203_826_HD116405_Filtre_None_bin1x1.438_red_spectrum.fits
 read file 342) : T1M_20190216_050239_104_HD116405_Filtre_None_bin1x1.439_red_spectrum.fits
 read file 343) : T1M_20190216_050314_384_HD116405_Filtre_None_bin1x1.440_red_spectrum.fits
 read file 344) : T1M_20190216_050349_661_HD116405_Filtre_None_bin1x1.441_red_spectrum.fits
*************************************************************************************
*    >>>> Reject bad quality file 344) : T1M_20190216_050349_661_HD116405_Filtre_None_bin1x1.441_red_spectrum.fits
*************************************************************************************
 read file 345) : T1M_20190216_050424_940_HD116405_Filtre_None_bin1x1.442_red_spectrum.fits
*************************************************************************************
*    >>>> Reject bad quality file 345) : T1M_20190216_050424_940_HD116405_F